In [ ]:
!pip install google.generativeai

In [ ]:
import google.generativeai as genai


genai.configure(api_key="")  # Replace with your actual API key or use secrets

# System prompts
interviewer_system = """You are an interviewer in a system design interview.
- Your role: Ask one clear, probing system design question per turn.
- Keep questions short and challenging.
- Focus on tradeoffs, scalability, and edge cases.
- Format output strictly as:
Interviewer: <your question here>
"""

candidate_system = """You are a candidate in a system design interview.
- Your role: Answer thoughtfully, but be realistic.
- Behaviors to include:
   * Sometimes hesitate ("Hmm, let me think...")
   * Occasionally give an incorrect answer, then correct yourself.
   * Sometimes be brilliant with sharp insights.
- Keep answers 2–5 sentences long.
- Format output strictly as:
Candidate: <your answer here>
"""

# Initialize models
interviewer = genai.GenerativeModel("gemini-2.5-flash", system_instruction=interviewer_system)
candidate = genai.GenerativeModel("gemini-2.5-flash", system_instruction=candidate_system)

def ask_model(model, conversation):
    response = model.generate_content(contents=conversation)
    return response.text.strip()

# Example conversation history to guide the models
history = [
    "Interviewer: Let’s begin. How would you design a URL shortener like bit.ly?",
    "Candidate: Hmm… okay, so the basic idea is we take a long URL, store it in a database, and generate a short code for it. Then when a user hits that short code, we redirect them back to the original URL.",
    "Interviewer: That works for a small setup. But let’s say you need to handle hundreds of millions of URLs. How would you scale it?",
    "Candidate: Uh, right… a single relational database probably won’t scale. Maybe… hmm… we could use sharding by hash of the URL? Or actually, maybe a distributed key-value store, like Cassandra or DynamoDB, would make more sense since we just need fast lookups.",
]

# Strictly alternate turns: interviewer then candidate
turn = "interviewer"  
for _ in range(5):
    if turn == "interviewer":
        interviewer_input = history[-1]
        interviewer_msg = ask_model(interviewer, interviewer_input)
        history.append(interviewer_msg)
        turn = "candidate"
    else:
        candidate_input = history[-1]
        candidate_msg = ask_model(candidate, candidate_input)
        history.append(candidate_msg)
        turn = "interviewer"

# Print transcript
print("\n".join(history))